In [19]:
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import tensorflow as tf

from functools import partial

# Plan
- Build Facial Detection model first
- Then build Facial Recognition model

## Data

Plan for data: 
1. Get a dataset of just faces
2. For Facial Detection model, combine this dataset with the ImageNet dataset
3. For Facial Recognition model, combine face dataset with pictures of approved faces(ie. mine) 

In [2]:
DATAPATH = '/home/misha/Desktop/data/face_recognition_Data'

In [3]:
notFace_path = DATAPATH + '/Not Face'

In [4]:
### Data clean up functionality
for obj_folder in os.listdir(notFace_path):
    if not os.path.isdir(notFace_path + f'/{obj_folder}'):
        continue
    else:
        print(f"Found folder {obj_folder}")
        for image_path in os.listdir(notFace_path + f'/{obj_folder}'):
            try:
                shutil.move(notFace_path + f'/{obj_folder}/{image_path}', notFace_path)
            except Exception as err:
                print(f"Error with file: {image_path}")
                print(err)

In [20]:
df = tf.keras.utils.image_dataset_from_directory(
    DATAPATH, 
    labels='inferred',
    color_mode='rgb',
    batch_size=24, 
    label_mode='binary',   ## Vector Representation (Use categorical_crossentropy loss)
    image_size=(640, 480),
    crop_to_aspect_ratio=True,
    seed=1, 
    shuffle=True
)

Found 3937 files belonging to 2 classes.


In [21]:
df

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 640, 480, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [22]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal', seed=1),
    tf.keras.layers.RandomRotation(0.5, seed=1)
])

In [23]:
aug_df = df.map(lambda x, y: (data_augmentation(x), y))

In [24]:
df = df.concatenate(aug_df).prefetch(1)

In [25]:
df

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 640, 480, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [26]:
DATASET_SIZE = len(df)
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15 

In [27]:
buffer_size = DATASET_SIZE
seed = 101
df = df.shuffle(buffer_size, seed=seed)

In [28]:
train_size = int(TRAIN_RATIO * DATASET_SIZE)
val_size = int(VAL_RATIO * DATASET_SIZE)
test_size = DATASET_SIZE - train_size - val_size 

In [29]:
train_df = df.take(train_size)
val_df = df.skip(train_size).take(val_size)
test_df = df.skip(train_size + val_size) # Takes the rest

In [30]:
len(val_df)

49

## Face Detection Model

In [32]:
DefaultConv2d = partial(tf.keras.layers.Conv2D, kernel_size=3, padding='same',
                       activation='relu', kernel_initializer='he_normal')

face_detect_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=[640,480,3]),
    DefaultConv2d(filters=64, kernel_size=7),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2d(filters=128),
    DefaultConv2d(filters=128),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2d(filters=256),
    DefaultConv2d(filters=256),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
face_detect_model.compile(loss=tf.keras.losses.BinaryCrossentropy, optimizer='nadam',
                         metrics=['accuracy'])